In [1]:
import json
from util import sanitize_filename
import os
from collections import defaultdict
from llm import chat_openai
from datetime import datetime

In [2]:
import os

chunk_size = 4000 # How many characters to read per chunk in website text
overlap = 25 # How much overlap between chunks
MAX_TOKENS = 40000 # 10K -- Roughly 3K tokens, $0.10 per MAX_TOKENs, max tokens to read per key question -- 40K tokens is $0.40
search_query = "How efficiently do the ECR enzymes work in Kitsatospor setae bacteria?"

In [3]:
# # Can be moved to util.py
# # Get the list of key questions
# def get_key_questions_list_string(search_query_file_safe):
#     file_path = f'autoscious_logs/{search_query_file_safe}/decompositions/improved_decomposition.json'

#     with open(file_path, 'r') as f:
#         question_decomposition = json.load(f)
#     key_questions = question_decomposition['key_drivers']['1']['hypotheses']['1']['key_questions']
#     print("\nkey_questions\n", key_questions)

#     numbered_key_questions_string = ""
#     for key, value in key_questions.items():
#         first_two_words = ' '.join(value.split())
#         numbered_key_questions_string += f'{int(key) - 1}. {first_two_words}\n'

#     print("\nnumbered_key_questions_string\n", numbered_key_questions_string)
#     return key_questions, numbered_key_questions_string

In [3]:
# 1) Get key questions
search_query_file_safe = sanitize_filename(search_query)
# key_questions_dict, key_questions_list_string = get_key_questions_list_string(search_query_file_safe)
# print("key_questions_dict", key_questions_dict)

# 2) create facts folder and subfolders
facts_folder_path = f'autoscious_logs/{sanitize_filename(search_query)}/facts'
if not os.path.exists(facts_folder_path):
    os.makedirs(facts_folder_path)
# for kq_idx in key_questions_dict:
#     kq_facts_folder_path = f'{facts_folder_path}/kq{int(kq_idx)-1}'
#     if not os.path.exists(kq_facts_folder_path):
#         os.makedirs(kq_facts_folder_path)
#     with open(f'{kq_facts_folder_path}/facts.txt', 'w') as f:
#         pass

In [10]:
# Can be broken to prompts.py and util.py
import os

def extract_facts_from_website_text(search_query_file_safe, key_question, website_title, website_text, website_url):
    seed_initial_question_decomposition_prompt = f'''
Key question: 
{key_question}

Context:
Ideally, I'm looking for a numerical answer.

Task:
What's the best answer based on the source text? Give me as specific and correct of an answer as possible. Then, quote the section of the source text that supports your answer. 

The output should be in JSON format: 
```json
{{
  "<insert best answer>": "<insert quote>"
}}
```

Source text: {website_text}

Respond only with the output, with no explanation or conversation.
'''
    # Ask GPT the prompt
    print("seed_initial_question_decomposition_prompt", seed_initial_question_decomposition_prompt)
    res = chat_openai(seed_initial_question_decomposition_prompt, model="gpt-3.5-turbo")
    print("Extracted quotes: ", res[0])

    # Save the quote to the corresponding key question index file
    res_json = json.loads(res[0])
    for key, value in res_json.items():
        answer = key
        quote = value

        # Only log if there is a quote
        if quote:
          file_name = f'autoscious_logs/{search_query_file_safe}/facts/facts.txt'

          with open(file_name, 'a', encoding='utf-8') as f:
              f.write(answer + os.linesep)

          # Save the best answer and quote into a JSON for reference retrieval later
          json_file_name = f'autoscious_logs/{search_query_file_safe}/facts/facts.json'
          if os.path.exists(json_file_name):
            with open(json_file_name, 'r', encoding='utf-8') as json_file:
                data = json.load(json_file)
          else:
              data = {}

          # Update the dictionary and save it back
          data[answer] = quote.replace('/"', '"') + f"[{website_url}]"
          
          with open(json_file_name, 'w', encoding='utf-8') as json_file:
              json.dump(data, json_file, ensure_ascii=False, indent=4)

    return

In [11]:
# import numpy as np
# from sklearn.metrics.pairwise import cosine_similarity
# from sentence_transformers import SentenceTransformer
# import re

# model = SentenceTransformer('all-MiniLM-L6-v2')  # or 'all-mpnet-base-v2'

# def get_most_relevant_chunks(key_question, text, CHUNK_SIZE):
#     # 1. Split text into chunks
#     chunks = [text[i:i+CHUNK_SIZE] for i in range(0, len(text), CHUNK_SIZE)]

#     # 2. For each chunk, compute the cosine similarity with key_question
#     key_embedding = model.encode([key_question])
#     chunk_embeddings = model.encode(chunks)
    
#     similarities = [cosine_similarity(key_embedding.reshape(1, -1), chunk_embedding.reshape(1, -1))[0][0] for chunk_embedding in chunk_embeddings]
    
#     # 3. Sort chunks by similarity
#     sorted_chunks = [chunk for _, chunk in sorted(zip(similarities, chunks), key=lambda pair: pair[0], reverse=True)]

#     # 4. Return top chunks
#     return sorted_chunks

In [12]:
# import torch
# from transformers import AutoTokenizer, AutoModel
# from sklearn.metrics.pairwise import cosine_similarity
# import re

# tokenizer = AutoTokenizer.from_pretrained("allenai/specter")
# model = AutoModel.from_pretrained("allenai/specter")

# def embed_text_using_specter(texts):
#     encoded_input = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors='pt')
#     with torch.no_grad():
#         model_output = model(**encoded_input)
#     embeddings = model_output.last_hidden_state.mean(dim=1)
#     return embeddings

# def get_most_relevant_chunks_specter(key_question, text, CHUNK_SIZE):
#     # 1. Split text into chunks
#     chunks = [text[i:i+CHUNK_SIZE] for i in range(0, len(text), CHUNK_SIZE)]

#     # 2. For each chunk, compute the cosine similarity with key_question
#     key_embedding = embed_text_using_specter([key_question])
#     chunk_embeddings = embed_text_using_specter(chunks)
    
#     # Note: Since the embeddings are in torch tensors, we'll use torch's cosine similarity function
#     similarities = [cosine_similarity(key_embedding, chunk_embedding.reshape(1, -1))[0][0] for chunk_embedding in chunk_embeddings]
    
#     # 3. Sort chunks by similarity
#     sorted_chunks = [chunk for _, chunk in sorted(zip(similarities, chunks), key=lambda pair: pair[0], reverse=True)]

#     # 4. Return top chunks
#     return sorted_chunks


In [13]:
from rank_bm25 import BM25Okapi
import re

def chunk_text(text: str, key_question: str, chunk_size: int, overlap: int = 0) -> list[str]:
    """
    Splits a text into overlapping chunks and ranks them using BM25 based on relevance to a key question.
    
    Args:
    - text (str): The source text.
    - key_question (str): The key question to rank the chunks by.
    - chunk_size (int): The size of each chunk.
    - num_chunk_samples (int): The number of top-ranked chunks to return.
    - overlap (int): The size of the overlap between chunks. Default is 0.
    
    Returns:
    - list[str]: The top-ranked chunks based on BM25 scores.
    """

    # 1. Split text into overlapping chunks
    chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size - overlap)]

    # 2. Tokenize the chunks
    tokenized_chunks = [re.findall(r"\w+", chunk) for chunk in chunks]

    # 3. Initialize BM25
    bm25 = BM25Okapi(tokenized_chunks)

    # 4. Query BM25 with the key question
    tokenized_question = re.findall(r"\w+", key_question)
    scores = bm25.get_scores(tokenized_question)

    # 5. Sort chunks by BM25 scores
    sorted_chunks = [chunk for _, chunk in sorted(zip(scores, chunks), key=lambda pair: pair[0], reverse=True)]

    # 6. Return top num_chunk_samples chunks
    return sorted_chunks


In [14]:
# # DEBUG

# # Go through each full text rated highly to extract facts from
# full_text_folder_path = f'autoscious_logs/{sanitize_filename(search_query)}/sources/full_text'

# keywords_query = "ECR enzymes efficiency Kitsatospor setae bacteria" # Hardcoding for now, but from research_machine_v2

# with open(r'C:\Users\1kevi\Desktop\projects\Research\autoscious-carbon-capture\question_answering\autoscious_logs\How_efficiently_do_the_ECR_enzymes_work_in_Kitsatospor_setae_bacteria_\sources\full_text\Intersubunit_Coupling_Enables_Fast_CO2_Fixation_by_Reductive_Carboxylases.txt', 'r', encoding='utf-8') as file:
#     title = file.readline().strip()
#     url = file.readline().strip()
#     text = file.read()

# # chunks = get_most_relevant_chunks_specter(keywords_query, text, chunk_size)
# chunks = chunk_text(text, keywords_query, chunk_size, overlap)
# for i, chunk in enumerate(chunks):
#     print("i: ", i)
#     print(chunk)
#     print()

In [19]:
with open(f'autoscious_logs/{search_query_file_safe}/sources/keywords_query.txt', 'r') as f:
    keywords_query = f.read().strip('"')

In [20]:
# This method can partially extract the answer, but not the exact table passage which is critical NOR does it prioritize the list of facts
# Go through each full text rated highly to extract facts from
full_text_folder_path = f'autoscious_logs/{sanitize_filename(search_query)}/sources/full_text'

# Loop through every file in the directory, just goes in order
for filename in os.listdir(full_text_folder_path):
    curr_tokens = 0
    if filename.endswith('.txt'):
        file_path = os.path.join(full_text_folder_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            title = file.readline().strip()
            url = file.readline().strip()
            text = file.read()
        
        # Break the text into chunk to extract information from
        chunks = chunk_text(text, keywords_query, chunk_size, overlap)
        for i, chunk in enumerate(chunks):
            print(f"Chunk: {i} / {len(chunks)}")
            extract_facts_from_website_text(search_query_file_safe, search_query, title, chunk, url)

            curr_tokens += len(chunk)
            if curr_tokens > MAX_TOKENS:
                print("Max tokens reached in chunks!")
                break

Chunk: 0 / 4
seed_initial_question_decomposition_prompt 
Key question: 
How efficiently do the ECR enzymes work in Kitsatospor setae bacteria?

Context:
Ideally, I'm looking for a numerical answer.

Task:
What's the best answer based on the source text? Give me as specific and correct of an answer as possible. Then, quote the section of the source text that supports your answer. 

The output should be in JSON format: 
```json
{
  "<insert best answer>": "<insert quote>"
}
```

Source text: This site needs JavaScript to work properly. Please enable it to take advantage of the complete set of features!
Clipboard, Search History, and several other advanced features are temporarily unavailable.
Skip to main page content
The .gov means it’s official.
Federal government websites often end in .gov or .mil. Before
sharing sensitive information, make sure you’re on a federal
government site.
The site is secure.
The https:// ensures that you are connecting to the
official website and that any in

### Rerank the facts.txt file based on relevancy

In [21]:
file_name = f'autoscious_logs/{search_query_file_safe}/facts/facts.txt'

with open(file_name, 'r', encoding='utf-8') as f:
    facts_list = f.read()

In [22]:
def get_rerank_facts_list_prompt(facts_list):
  return f'''
Key question: 
How efficiently do the ECR enzymes work in Kitsatospor setae bacteria?

Context:
Ideally, I'm looking for a numerical answer.

Task:
Rerank the following facts based on how well they answer the key question. The more specific and correct, the better. The best answer should be at the top, and the worst answer should be at the bottom of the list. Use direct quotes and do not change the wording of the facts. Leave out facts that are not relevant to the key question.

Current facts list:
{facts_list}

The output should be a JSON list of facts:
```json
['<insert fact>', etc.]
```

Respond only with the output, with no explanation or conversation.
'''
reranked_facts_list = json.loads(chat_openai(get_rerank_facts_list_prompt(facts_list), model="gpt-3.5-turbo")[0])
print(reranked_facts_list)

Prompt:  
Key question: 
How efficiently do the ECR enzymes work in Kitsatospor setae bacteria?

Context:
Ideally, I'm looking for a numerical answer.

Task:
Rerank the following facts based on how well they answer the key question. The more specific and correct, the better. The best answer should be at the top, and the worst answer should be at the bottom of the list. Use direct quotes and do not change the wording of the facts. Leave out facts that are not relevant to the key question.

Current facts list:
The ECR enzymes in Kitsatospor setae bacteria are highly efficient.

The ECR enzymes in Kitsatospor setae bacteria work efficiently in CO2 fixation.

highly efficient

The bacterial enzyme ECRs in Kitasatospora setae work 100 times faster than without the presence of molecular 'glue' and twist.

20 times faster than plant enzymes do during photosynthesis

ECRs are some of the most efficient CO2-fixing enzymes described to date.

The ECR enzymes in Kitsatospor setae bacteria work wi

In [23]:
file_name = f'autoscious_logs/{search_query_file_safe}/facts/facts.txt'

with open(file_name, 'w', encoding='utf-8') as f:
    for answer in reranked_facts_list:
        f.write(answer + os.linesep)

Stopped after one iteration of somewhat successfully extracting the best answer and knowing if it did. Trying to work on generating potentially impactful experiments in the generative_experimentation folder